In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

df = pd.read_excel(r' Graded_HW01_data.xlsx')
cols = ['SPY', 'XLB', 'XLE', 'XLF']
returns = df[cols].dropna()

def garch_log(paras, r):
    omega, alpha, beta = paras
    T = len(r)
    s = np.zeros(T)
    s[0] = np.var(r)
    for t in range(1, T):
        s[t] = omega + alpha * r[t-1]**2 + beta * s[t-1]
    log = -0.5 * np.sum(np.log(2 * np.pi * s) + r**2 / s)
    return -log

garch_paras = {}
for col in cols:
    r = returns[col].values
    garch_result = minimize(garch_log, [0.1, 0.1, 0.8], args=(r,), method='BFGS')
    garch_paras[col] = {'omega': garch_result.x[0], 'alpha': garch_result.x[1], 'beta': garch_result.x[2]}
    
def dcc_log(paras, r, garch_paras):
    alpha, beta = paras
    T = len(r)
    s = np.zeros((T, 4))
    corr = np.zeros((T, 4, 4))
    s[0,:] = np.array([garch_paras[col]['omega'] for col in cols])
    corr[0,:,:] = np.eye(4)
    for t in range(1, T):
        s[t,:] = (1 - alpha - beta) * s[t-1,:] + alpha * r[t-1,:]**2 + beta * s[t-1,:]
        std = np.sqrt(s[t,:])
        for i in range(4):
            for j in range(i+1, 4):
            corr[t,i,j] = corr[t-1,i,j] + alpha * (r[t-1,i] / std[i]) * (r[t-1,j] / std[j]) * (corr[t-1,i,j] - corr[t-1,j,i])
            corr[t,j,i] = corr[t,i,j]
        log = -0.5 * np.sum(np.log(1 - alpha - beta) + r**2 / s + np.log(s))
        return -log

r = returns.values
dcc_result = minimize(dcc_log, [0.01, 0.98], args=(r, garch_paras), method='BFGS')
dcc_paras = {'alpha': dcc_result.x[0], 'beta': dcc_result.x[1]}
for col, paras in garch_paras.items():
    print(col)
    print(f"omega = {paras['omega']:.4f}, alpha = {paras['alpha']:.4f}, beta = {paras['beta']:.4f}")
print("DCC")
print(f"alpha = {dcc_paras['alpha']:.4f}, beta = {dcc_paras['beta']:.4f}")